In [23]:
import numpy as np
import pandas as pd

In [24]:
file_name = 'supply_chain_logisitcs_problem.xlsx'
df_freight = pd.read_excel(file_name, sheet_name='FreightRates')
df_order = pd.read_excel(file_name, sheet_name='OrderList')
df_plant_ports = pd.read_excel(file_name, sheet_name='PlantPorts')
df_prod_per_plant = pd.read_excel(file_name, sheet_name='ProductsPerPlant')
df_vmi_cust = pd.read_excel(file_name, sheet_name='VmiCustomers')
df_wh_cap = pd.read_excel(file_name, sheet_name='WhCapacities')
df_wh_costs = pd.read_excel(file_name, sheet_name='WhCosts')

In [25]:
display(df_freight.head(3))
display(df_order.head(3))
display(df_plant_ports.head(3))
display(df_prod_per_plant.head(3))
display(df_vmi_cust.head(3))
display(df_wh_cap.head(3))
display(df_wh_costs.head(3))

,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,V444_6,PORT08,PORT09,250.0,499.99,DTD,43.2272,0.7132,AIR,2,V88888888_0
1,V444_6,PORT08,PORT09,65.0,69.99,DTD,43.2272,0.7512,AIR,2,V88888888_0
2,V444_6,PORT08,PORT09,60.0,64.99,DTD,43.2272,0.7892,AIR,2,V88888888_0


,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20


,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03


,Plant Code,Product ID
0,PLANT15,1698815
1,PLANT17,1664419
2,PLANT17,1664426


,Plant Code,Customers
0,PLANT02,V5555555555555_16
1,PLANT02,V555555555555555_29
2,PLANT02,V555555555_3


,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111


,WH,Cost/unit
0,PLANT15,1.415063
1,PLANT17,0.428947
2,PLANT18,2.036254


In [31]:
df = pd.merge(df_order[['Order Date','Order ID','Carrier','Origin Port','Destination Port','Unit quantity','Weight']], df_plant_ports, left_on = ['Origin Port'], right_on = ['Port']).rename(columns = {'Plant Code' : 'origin_plant_code'}).drop(['Port'], axis = 1)
df = pd.merge(df, df_plant_ports, left_on = ['Destination Port'], right_on = ['Port']).rename(columns = {'Plant Code' : 'destination_plant_code'}).drop(['Port'], axis = 1)
df = pd.merge(df, df_freight, left_on = ['Carrier', 'Origin Port', 'Destination Port'], right_on = ['Carrier', 'orig_port_cd', 'dest_port_cd'])
df.head()

,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,origin_plant_code,destination_plant_code,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,2013-05-26,1.447385e+09,V444_0,PORT09,PORT09,332,1.023143,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0
1,2013-05-26,1.447385e+09,V444_0,PORT09,PORT09,332,1.023143,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTD,31.2784,13.2784,GROUND,0,V88888888_0
2,2013-05-26,1.447338e+09,V444_0,PORT09,PORT09,358,0.613886,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0
3,2013-05-26,1.447338e+09,V444_0,PORT09,PORT09,358,0.613886,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTD,31.2784,13.2784,GROUND,0,V88888888_0
4,2013-05-26,1.447407e+09,V444_0,PORT09,PORT09,409,37.041956,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0


In [40]:
df_freight_grouped = df_freight.groupby(['Carrier','orig_port_cd','dest_port_cd']).agg({
    'minm_wgh_qty' : lambda x : sum(x)/len(x),
    'max_wgh_qty' : lambda x : sum(x)/len(x),
    'rate' : lambda x : sum(x)/len(x),
    'minimum cost' : lambda x : sum(x)/len(x)
}).reset_index().rename(columns = {'minimum cost' : 'min_cost','minm_wgh_qty' : 'min_wgh_qty'})
display(df_freight_grouped.head())

,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000,20569.990,0.08448,23.83840
1,V444_0,PORT03,PORT09,0.000,5000.000,12.69420,28.94420
2,V444_0,PORT04,PORT09,570.000,20569.990,0.05200,2.47720
3,V444_0,PORT09,PORT09,0.000,5000.000,12.77840,31.27840
4,V444_1,PORT02,PORT09,197.556,1190.745,0.29114,23.55734


In [41]:
df_cost = df_freight_grouped.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
df_cost = round(df_cost, 0)
df_cost = df_cost[df_cost.index.isin(df['Origin Port'])]
array_cost_pulp = df_cost.to_numpy().tolist()
array_cost_pulp

[[2.0], [24.0]]

In [42]:
df_cost

dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT09,24.0
